## Prioritized Experience Replay
In the original DQN paper (and implementation), random transitions were sampled uniformly from the replay buffer and used for learning. Naturally the next question to ask is, "can we sample some other way and expect better performance?" That is, can we learn more from some transitions than from others? To draw a parallel, envision some task you're accustomed to performing with an expected outcome. If someday you perform this same task but achieve some wildly different outcome, you'd stop to reevaluate the situation. You'd think long and hard trying to come up with an explanation (adjusting your model) for this unexpected event. This is same idea behind prioritized experience replay. We assign some priority to transitions and when it's time to sample, we choose the transitions with the highest priorities because we expect to learn more from them.

In [1], the TD error of a transition is used to compute its priority. The authors stated that it "indicates how surprising or unexpected the transition is." Two methods for assigning priority are given:

Proportional Prioritization: $p_i = |\delta_i| + \epsilon$

Rank Prioritization: $p_i = \frac{1}{rank(i)}$

where $\delta_i$ is the TD error, $\epsilon$ is a small constant and rank(i) is the rank of transition i when the replay buffer is sorted according to $|\delta_i|$

For rank proritization, a priority queue can be used. For proportional prioritization, the authors have used the sum tree data structure with the property that the value of all internal nodes are the sum of their respective children. Thus, the root node's value is the sum of all leaf nodes. See SumTree for an understanding of how the sum tree works. Below I compare the performance of an agent on the CartPole problem using uniform sampling and with prioritized sampling.

### References 
[1] Tom Schaul, David Silver et al (2016). Prioritized Experience Replay. https://arxiv.org/abs/1511.05952